In [3]:
import numpy as np
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from src.utils.text_preprocessing import preprocess_text, tokenize
from src.utils.reporting import get_cross_validation_report
from src.utils.vector_space_analysis import *
from src.utils.common import *
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv('data/reviews_excerpt.csv')
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    df['text_pp'] = df['text'].progress_apply(lambda row: preprocess_text(row, removing_stopwords=False))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Users\Dmitry.Volkov\Anaconda3\envs\support-cases-2\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████| 12230/12230 [00:03<00:00, 3739.61it/s]


In [13]:
from flair.embeddings import WordEmbeddings, DocumentRNNEmbeddings
from flair.data import Sentence

glove_embedding = WordEmbeddings('glove')
document_rnn_embeddings = DocumentRNNEmbeddings([glove_embedding])

def rnn_vectorization(input_array):
    sentences = [Sentence(text) for text in input_array]
    for sentence in sentences:
        document_rnn_embeddings.embed(sentence)
    return np.array([sentence.get_embedding().detach().numpy() for sentence in sentences])

In [17]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.preprocessing import FunctionTransformer
from IPython.display import display

X, y = df['text_pp'], df['score'].to_numpy()

weighted_f1, report_df, confusion_df = get_cross_validation_report(
    X, y,
    model_factory=lambda: Pipeline([
        ('embd', FunctionTransformer(func=rnn_vectorization)),
        ('smote', SMOTE(random_state=0)),
        ('svc', SVC()),
    ]),
    seed=0
)
print(weighted_f1)
display(report_df)
display(confusion_df)

100%|██████████| 5/5 [22:06<00:00, 265.39s/it]

0.2888


,precision,recall,f1,support
1.0,0.335985,0.362633,0.348801,2446.0
2.0,0.268890,0.260425,0.264590,2446.0
3.0,0.254223,0.246116,0.250104,2446.0
4.0,0.252166,0.226083,0.238413,2446.0
5.0,0.328195,0.356909,0.341951,2446.0


,Pred 1.0,Pred 2.0,Pred 3.0,Pred 4.0,Pred 5.0
True 1.0,887,531,392,303,333
True 2.0,582,637,489,378,360
True 3.0,453,458,602,482,451
True 4.0,351,399,500,553,643
True 5.0,367,344,385,477,873


In [18]:
from flair.embeddings import TransformerDocumentEmbeddings

roberta_document_embeddings = TransformerDocumentEmbeddings('roberta-base')

def roberta_transformer_vectorization(input_array):
    sentences = [Sentence(text) for text in input_array]
    for sentence in sentences:
        roberta_document_embeddings.embed(sentence)
    return np.array([sentence.get_embedding().detach().numpy() for sentence in sentences])

In [19]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.preprocessing import FunctionTransformer
from IPython.display import display

X, y = df['text_pp'], df['score'].to_numpy()

weighted_f1, report_df, confusion_df = get_cross_validation_report(
    X, y,
    model_factory=lambda: Pipeline([
        ('embd', FunctionTransformer(func=roberta_transformer_vectorization)),
        ('smote', SMOTE(random_state=0)),
        ('svc', SVC()),
    ]),
    seed=0
)
print(weighted_f1)
display(report_df)
display(confusion_df)

  0%|          | 0/5 [41:41<?, ?it/s]

KeyboardInterrupt

